In [5]:

import random as pyrandom
from sage.all import *
import pandas as pd
import os
import time

# ============================================================
#  SUBPATH NUMBER (OPTIMIZIRANO)
# ============================================================

def subpath_number(G):
    V = list(G.vertices())
    idx = {v: i for i, v in enumerate(V)}
    n = len(V)

    counts = [[0] * n for _ in range(n)]

    for s in V:
        s_idx = idx[s]

        def dfs(u, visited):
            u_idx = idx[u]
            counts[s_idx][u_idx] += 1

            for nei in G.neighbors(u):
                if nei not in visited:
                    visited.add(nei)
                    dfs(nei, visited)
                    visited.remove(nei)

        visited = set([s])
        dfs(s, visited)

    total = 0
    for i in range(n):
        for j in range(i, n):
            total += counts[i][j]

    return total


# ============================================================
#  SHARANJE OPTIMUMA v CSV (popolnoma popravljeno)
# ============================================================

def save_to_csv(n, mu, graph, score, direction):
    filename = "rezultati_poskus_n_8_N300.csv"

    g6 = graph.graph6_string()

    # Če CSV ne obstaja → kreiraj
    if not os.path.exists(filename):
        df = pd.DataFrame(columns=[
            "n",
            "µ(G)",
            "min p_n(G)",
            "max p_n(G)",
            "graph6_min",
            "graph6_max"
        ])
        df.to_csv(filename, index=False)

    df = pd.read_csv(filename, encoding="utf-8")

    # poskrbimo, da stolpci obstajajo (če imaš kak star CSV)
    for col in ["min p_n(G)", "max p_n(G)", "graph6_min", "graph6_max"]:
        if col not in df.columns:
            df[col] = pd.NA

    df["n"] = df["n"].astype(int)
    df["µ(G)"] = df["µ(G)"].astype(int)

    n_int = int(n)
    mu_int = int(mu)

    mask = (df["n"] == n_int) & (df["µ(G)"] == mu_int)

    # 1) Če vrstice NI → nova
    if not mask.any():
        new_row = {
            "n": n_int,
            "µ(G)": mu_int,
            "min p_n(G)": score if direction == "min" else pd.NA,
            "max p_n(G)": score if direction == "max" else pd.NA,
            "graph6_min": g6 if direction == "min" else pd.NA,
            "graph6_max": g6 if direction == "max" else pd.NA
        }

        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
        df.to_csv(filename, index=False)
        return

    # 2) Če vrstica OBSTAJA → update
    idx = df.index[mask].tolist()[0]

    if direction == "min":
        old = df.at[idx, "min p_n(G)"]

        if pd.isna(old) or old == "":
            # še nič ni zapisano → zapišemo
            df.at[idx, "min p_n(G)"] = score
            df.at[idx, "graph6_min"] = g6
        else:
            # že obstaja številka → zamenjamo samo, če je nova boljša (manjša)
            if score < float(old):
                df.at[idx, "min p_n(G)"] = score
                df.at[idx, "graph6_min"] = g6

    else:  # direction == "max"
        old = df.at[idx, "max p_n(G)"]

        if pd.isna(old) or old == "":
            df.at[idx, "max p_n(G)"] = score
            df.at[idx, "graph6_max"] = g6
        else:
            if score > float(old):
                df.at[idx, "max p_n(G)"] = score
                df.at[idx, "graph6_max"] = g6

    df.to_csv(filename, index=False)



# ============================================================
#  MUTATE GRAPH
# ============================================================

def mutate_graph(G):
    H = G.copy()

    edges = list(H.edges())
    non_edges = [(u, v) for u in H.vertices()
                 for v in H.vertices()
                 if u < v and not H.has_edge(u, v)]

    if not edges or not non_edges:
        return H

    e_remove = pyrandom.choice(edges)
    H.delete_edge(e_remove)

    e_add = pyrandom.choice(non_edges)
    H.add_edge(e_add)

    if not H.is_connected():
        return G

    return H


# ============================================================
#  DODAJ 1 VOZLIŠČE
# ============================================================

def add_one_vertex_and_connect(G_old, direction):
    H = Graph(G_old)

    verts = H.vertices()
    new_v = max(verts) + 1
    H.add_vertex(new_v)

    degrees = {v: H.degree(v) for v in H.vertices() if v != new_v}

    if direction == "min":
        anchor = min(degrees, key=degrees.get)
    else:
        anchor = max(degrees, key=degrees.get)

    H.add_edge(anchor, new_v)
    return H


# ============================================================
#  DODAJ 1 POVEZAVO (najbližji non-edge)
# ============================================================

def add_one_edge(G_old):
    H = Graph(G_old)

    V = H.vertices()
    for i in range(len(V)):
        for j in range(i+1, len(V)):
            u, v = V[i], V[j]
            if not H.has_edge(u, v):
                H.add_edge(u, v)
                return H

    raise ValueError("Graf je že poln.")


# ============================================================
#  LOAD G(n,µ) IZ TVOJEGA CSV — POPRAVLJENO
# ============================================================

def load_G(n_val, mu_val, direction):
    df = pd.read_csv("rezultati_poskus_n_8_N300.csv", encoding="utf-8")

    df["n"] = df["n"].astype(int)
    df["µ(G)"] = df["µ(G)"].astype(int)

    # Konverzija Sage Integer → Python int
    n_val = int(n_val)
    mu_val = int(mu_val)

    subset = df[df["n"] == n_val]
    if subset.empty:
        raise ValueError(f"Ni vrstic za n={n_val}")

    row = subset[subset["µ(G)"] == mu_val]
    if row.empty:
        raise ValueError(f"Ni vrstice za n={n_val}, µ={mu_val}")

    # KLJUČNO — prisili Python int indeks
    row = row.iloc[int(0)]      # ← EDINA PRAVA OBLIKA

    # graph6
    if direction == "min":
        g6 = str(row["graph6_min"]).strip()
    else:
        g6 = str(row["graph6_max"]).strip()

    if g6 == "":
        raise ValueError(f"Prazni graph6 zapis pri n={n_val}, µ={mu_val}")

    return Graph(g6)



# ============================================================
#  SIMULATED ANNEALING
# ============================================================

def simulated_annealing(n, m, direction,
                        T_start=3.0, T_end=0.001,
                        cooling=0.99, max_steps=N,
                        initial_graph=None):

    G = Graph(initial_graph)

    best_G = G.copy()
    best_score = subpath_number(G)

    current_G = G.copy()
    current_score = best_score

    T = T_start

    for step in range(max_steps):
        new_G = mutate_graph(current_G)
        new_score = subpath_number(new_G)

        if direction == "min":
            delta = new_score - current_score
        else:
            delta = current_score - new_score

        if delta < 0:
            current_G = new_G
            current_score = new_score
        else:
            if pyrandom.random() < exp(-delta/T):
                current_G = new_G
                current_score = new_score

        improved = (
            (direction == "min" and current_score < best_score) or
            (direction == "max" and current_score > best_score)
        )

        if improved:
            best_G = current_G.copy()
            best_score = current_score

        T *= cooling
        if T < T_end:
            break

    return best_G, best_score


# ============================================================
#  GONILNA FUNKCIJA
# ============================================================

def compute_all_for_n(direction, n, N):

    mu_max = ((n-1)*(n-2)/2)
    mu_max_prev = ((n-2)*(n-3)/2)
    

    for mu in range(0, mu_max + 1):

        print(f"Obdelujem: n={n}, µ={mu}")

        if mu <= mu_max_prev:
            G8 = load_G(n-1, mu, direction)
            Gstart = add_one_vertex_and_connect(G8, direction)
        else:
            Gprev = load_G(n, mu-1, direction)
            Gstart = add_one_edge(Gprev)

        m = mu + n - 1

        best_graph, best_value = simulated_annealing(
            n=n,
            m=m,
            direction=direction,
            initial_graph=Gstart,
            max_steps=N
        )

        print(f"=== Rezultati za n={n}, µ={mu} ===")
        print("Best value:", best_value)
        print("graph6:", best_graph.graph6_string())

        save_to_csv(n, mu, best_graph, best_value, direction)


# ============================================================
#  START
# ============================================================

compute_all_for_n(direction="min", n=8, N=300)


Obdelujem: n=8, µ=0
=== Rezultati za n=8, µ=0 ===
Best value: 36
graph6: G??F{?
Obdelujem: n=8, µ=1
=== Rezultati za n=8, µ=1 ===
Best value: 49
graph6: G?AFy?
Obdelujem: n=8, µ=2


/tmp/ipykernel_72720/1075849697.py:101: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '36' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, "min p_n(G)"] = score
/tmp/ipykernel_72720/1075849697.py:101: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '49' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, "min p_n(G)"] = score
/tmp/ipykernel_72720/1075849697.py:101: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '66' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, "min p_n(G)"] = score


=== Rezultati za n=8, µ=2 ===
Best value: 66
graph6: G?`Fx?
Obdelujem: n=8, µ=3
=== Rezultati za n=8, µ=3 ===
Best value: 87
graph6: GCOf[_
Obdelujem: n=8, µ=4


/tmp/ipykernel_72720/1075849697.py:101: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '87' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, "min p_n(G)"] = score
/tmp/ipykernel_72720/1075849697.py:101: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '127' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, "min p_n(G)"] = score


=== Rezultati za n=8, µ=4 ===
Best value: 127
graph6: G@`FyC
Obdelujem: n=8, µ=5
=== Rezultati za n=8, µ=5 ===
Best value: 207
graph6: G?Bz{G
Obdelujem: n=8, µ=6


...

/tmp/ipykernel_72720/1075849697.py:101: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '32949' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, "min p_n(G)"] = score


=== Rezultati za n=8, µ=20 ===
Best value: 43061
graph6: G~~~~w
Obdelujem: n=8, µ=21


/tmp/ipykernel_72720/1075849697.py:101: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '43061' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, "min p_n(G)"] = score


=== Rezultati za n=8, µ=21 ===
Best value: 54804
graph6: G~~~~{


/tmp/ipykernel_72720/1075849697.py:101: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '54804' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, "min p_n(G)"] = score


In [8]:

import random as pyrandom
from sage.all import *
import pandas as pd
import os
import time

# ============================================================
#  SUBPATH NUMBER (OPTIMIZIRANO)
# ============================================================

def subpath_number(G):
    V = list(G.vertices())
    idx = {v: i for i, v in enumerate(V)}
    n = len(V)

    counts = [[0] * n for _ in range(n)]

    for s in V:
        s_idx = idx[s]

        def dfs(u, visited):
            u_idx = idx[u]
            counts[s_idx][u_idx] += 1

            for nei in G.neighbors(u):
                if nei not in visited:
                    visited.add(nei)
                    dfs(nei, visited)
                    visited.remove(nei)

        visited = set([s])
        dfs(s, visited)

    total = 0
    for i in range(n):
        for j in range(i, n):
            total += counts[i][j]

    return total


# ============================================================
#  SHARANJE OPTIMUMA v CSV (popolnoma popravljeno)
# ============================================================

def save_to_csv(n, mu, graph, score, direction):
    filename = "rezultati_poskus_n_8_N2000_T6.csv"

    g6 = graph.graph6_string()

    # Če CSV ne obstaja → kreiraj
    if not os.path.exists(filename):
        df = pd.DataFrame(columns=[
            "n",
            "µ(G)",
            "min p_n(G)",
            "max p_n(G)",
            "graph6_min",
            "graph6_max"
        ])
        df.to_csv(filename, index=False)

    df = pd.read_csv(filename, encoding="utf-8")

    # poskrbimo, da stolpci obstajajo (če imaš kak star CSV)
    for col in ["min p_n(G)", "max p_n(G)", "graph6_min", "graph6_max"]:
        if col not in df.columns:
            df[col] = pd.NA

    df["n"] = df["n"].astype(int)
    df["µ(G)"] = df["µ(G)"].astype(int)

    n_int = int(n)
    mu_int = int(mu)

    mask = (df["n"] == n_int) & (df["µ(G)"] == mu_int)

    # 1) Če vrstice NI → nova
    if not mask.any():
        new_row = {
            "n": n_int,
            "µ(G)": mu_int,
            "min p_n(G)": score if direction == "min" else pd.NA,
            "max p_n(G)": score if direction == "max" else pd.NA,
            "graph6_min": g6 if direction == "min" else pd.NA,
            "graph6_max": g6 if direction == "max" else pd.NA
        }

        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
        df.to_csv(filename, index=False)
        return

    # 2) Če vrstica OBSTAJA → update
    idx = df.index[mask].tolist()[0]

    if direction == "min":
        old = df.at[idx, "min p_n(G)"]

        if pd.isna(old) or old == "":
            # še nič ni zapisano → zapišemo
            df.at[idx, "min p_n(G)"] = score
            df.at[idx, "graph6_min"] = g6
        else:
            # že obstaja številka → zamenjamo samo, če je nova boljša (manjša)
            if score < float(old):
                df.at[idx, "min p_n(G)"] = score
                df.at[idx, "graph6_min"] = g6

    else:  # direction == "max"
        old = df.at[idx, "max p_n(G)"]

        if pd.isna(old) or old == "":
            df.at[idx, "max p_n(G)"] = score
            df.at[idx, "graph6_max"] = g6
        else:
            if score > float(old):
                df.at[idx, "max p_n(G)"] = score
                df.at[idx, "graph6_max"] = g6

    df.to_csv(filename, index=False)



# ============================================================
#  MUTATE GRAPH
# ============================================================

def mutate_graph(G):
    H = G.copy()

    edges = list(H.edges())
    non_edges = [(u, v) for u in H.vertices()
                 for v in H.vertices()
                 if u < v and not H.has_edge(u, v)]

    if not edges or not non_edges:
        return H

    e_remove = pyrandom.choice(edges)
    H.delete_edge(e_remove)

    e_add = pyrandom.choice(non_edges)
    H.add_edge(e_add)

    if not H.is_connected():
        return G

    return H


# ============================================================
#  DODAJ 1 VOZLIŠČE
# ============================================================

def add_one_vertex_and_connect(G_old, direction):
    H = Graph(G_old)

    verts = H.vertices()
    new_v = max(verts) + 1
    H.add_vertex(new_v)

    degrees = {v: H.degree(v) for v in H.vertices() if v != new_v}

    if direction == "min":
        anchor = min(degrees, key=degrees.get)
    else:
        anchor = max(degrees, key=degrees.get)

    H.add_edge(anchor, new_v)
    return H


# ============================================================
#  DODAJ 1 POVEZAVO (najbližji non-edge)
# ============================================================

def add_one_edge(G_old):
    H = Graph(G_old)

    V = H.vertices()
    for i in range(len(V)):
        for j in range(i+1, len(V)):
            u, v = V[i], V[j]
            if not H.has_edge(u, v):
                H.add_edge(u, v)
                return H

    raise ValueError("Graf je že poln.")


# ============================================================
#  LOAD G(n,µ) IZ TVOJEGA CSV — POPRAVLJENO
# ============================================================

def load_G(n_val, mu_val, direction):
    df = pd.read_csv("rezultati_poskus_n_8_N2000_T6.csv", encoding="utf-8")

    df["n"] = df["n"].astype(int)
    df["µ(G)"] = df["µ(G)"].astype(int)

    # Konverzija Sage Integer → Python int
    n_val = int(n_val)
    mu_val = int(mu_val)

    subset = df[df["n"] == n_val]
    if subset.empty:
        raise ValueError(f"Ni vrstic za n={n_val}")

    row = subset[subset["µ(G)"] == mu_val]
    if row.empty:
        raise ValueError(f"Ni vrstice za n={n_val}, µ={mu_val}")

    # KLJUČNO — prisili Python int indeks
    row = row.iloc[int(0)]      # ← EDINA PRAVA OBLIKA

    # graph6
    if direction == "min":
        g6 = str(row["graph6_min"]).strip()
    else:
        g6 = str(row["graph6_max"]).strip()

    if g6 == "":
        raise ValueError(f"Prazni graph6 zapis pri n={n_val}, µ={mu_val}")

    return Graph(g6)



# ============================================================
#  SIMULATED ANNEALING
# ============================================================

def simulated_annealing(n, m, direction,
                        T_start=6, T_end=0.002,
                        cooling=0.9965, max_steps=N,
                        initial_graph=None):

    G = Graph(initial_graph)

    best_G = G.copy()
    best_score = subpath_number(G)

    current_G = G.copy()
    current_score = best_score

    T = T_start

    for step in range(max_steps):
        new_G = mutate_graph(current_G)
        new_score = subpath_number(new_G)

        if direction == "min":
            delta = new_score - current_score
        else:
            delta = current_score - new_score

        if delta < 0:
            current_G = new_G
            current_score = new_score
        else:
            if pyrandom.random() < exp(-delta/T):
                current_G = new_G
                current_score = new_score

        improved = (
            (direction == "min" and current_score < best_score) or
            (direction == "max" and current_score > best_score)
        )

        if improved:
            best_G = current_G.copy()
            best_score = current_score

        T *= cooling
        if T < T_end:
            break

    return best_G, best_score


# ============================================================
#  GONILNA FUNKCIJA
# ============================================================

def compute_all_for_n(direction, n, N):

    mu_max = ((n-1)*(n-2)/2)
    mu_max_prev = ((n-2)*(n-3)/2)
    

    for mu in range(0, mu_max + 1):

        print(f"Obdelujem: n={n}, µ={mu}")

        if mu <= mu_max_prev:
            G8 = load_G(n-1, mu, direction)
            Gstart = add_one_vertex_and_connect(G8, direction)
        else:
            Gprev = load_G(n, mu-1, direction)
            Gstart = add_one_edge(Gprev)

        m = mu + n - 1

        best_graph, best_value = simulated_annealing(
            n=n,
            m=m,
            direction=direction,
            initial_graph=Gstart,
            max_steps=N
        )

        print(f"=== Rezultati za n={n}, µ={mu} ===")
        print("Best value:", best_value)
        print("graph6:", best_graph.graph6_string())

        save_to_csv(n, mu, best_graph, best_value, direction)


# ============================================================
#  START
# ============================================================

compute_all_for_n(direction="min", n=8, N=2000)


Obdelujem: n=8, µ=0
=== Rezultati za n=8, µ=0 ===
Best value: 36
graph6: G??F{?
Obdelujem: n=8, µ=1
=== Rezultati za n=8, µ=1 ===
Best value: 49
graph6: G?AFy?
Obdelujem: n=8, µ=2
=== Rezultati za n=8, µ=2 ===
Best value: 66
graph6: G?`Fx?
Obdelujem: n=8, µ=3
=== Rezultati za n=8, µ=3 ===
Best value: 87
graph6: GeGBWK
Obdelujem: n=8, µ=4
=== Rezultati za n=8, µ=4 ===
Best value: 127
graph6: GAIFwK
Obdelujem: n=8, µ=5
=== Rezultati za n=8, µ=5 ===
Best value: 207
graph6: G?J}gK
Obdelujem: n=8, µ=6
=== Rezultati za n=8, µ=6 ===
Best value: 279
graph6: G?B~wK
Obdelujem: n=8, µ=7
=== Rezultati za n=8, µ=7 ===
Best value: 426
graph6: G@B~wK
Obdelujem: n=8, µ=8
=== Rezultati za n=8, µ=8 ===
Best value: 653
graph6: GGB~wk
Obdelujem: n=8, µ=9
=== Rezultati za n=8, µ=9 ===
Best value: 1005
graph6: G?~|wW
Obdelujem: n=8, µ=10
=== Rezultati za n=8, µ=10 ===
Best value: 1636
graph6: GTm~y?
Obdelujem: n=8, µ=11
=== Rezultati za n=8, µ=11 ===
Best value: 1964
graph6: Gsb~{K
Obdelujem: n=8, µ=12
=== 

KeyboardInterrupt: 